In [ ]:
#подключение к диску
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#импорт библиотек и модулей
import pandas as pd
from string import punctuation as p
from string import ascii_letters as al
from math import log

#путь к файлу
path = '/content/gdrive/My Drive/COLAB_Andemir/homeworks/spam.csv'

#несущественные слова
incons = ["the","all","off","of","or","but","and","through","though","although","then",
          "not","in","out","on","about","too","yet","nor","either","neither","so","therefore",
          "moreover","furthermore","however","also","hence","to","at","from","with","by","as",
          "this","that","these","those","between","only","for","a","an","into","non"," no","yes",
          "up","down","even","ever","am","is","are","was","were","will","i","he","she","we","they",
          "it","you","my","its","his","her","their","our","your","have","has","be","been","do","does",
          "not","thus","would","could","can","until","him","me","them","us","if","unless","who","when",
          "where","which","whether","what","why","whoever","whatever","whom","had","away","did","there",
          "whose","more","most","co","re","la","le","any","other","each","much","than","some","every",
          "thing","else","one","two","three","four","five","six","seven","eight","nine","ten",
           "", "!", ",", "-", ".", "+", "=", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "don't", "it's", "s", "o", "t", "ve"]

#читаем файл
df = pd.read_csv(path)
df = df.fillna('')

#удаляем лишние столбцы
df['v2'] = df['v2'] + df['Unnamed: 2'] + df['Unnamed: 3'] + df['Unnamed: 4']
df.drop(['Unnamed: 2', 'Unnamed: 3','Unnamed: 4'], axis=1, inplace=True)

#игнорируем короткие предложения
df = pd.DataFrame(i for _, i in df.iterrows() if len(i['v2'].split()) > 3)

#разделяем данные
proportion = round(len(df) * 0.8)
train = df[:proportion]
test = df[proportion:]

delete = 3
#создаем списки spam и ham слов
spam_l = [j.lower().strip(p) for _, i in train.iterrows() if i['v1'] == 'spam' for j in i['v2'].split() \
             if j.lower().strip(p) not in incons and len(j.strip(p)) >= delete and \
             len(set(j.strip(p)) & set(p)) == 0 and len(set(j.strip(p)).difference(al)) == 0]

ham_l = [j.lower().strip(p) for i in train[train['v1'] == 'ham']['v2'] for j in i.split() \
            if j.lower().strip(p) not in incons and len(j.strip(p)) >= delete \
            and len(set(j.strip(p)) & set(p)) == 0 and len(set(j.strip(p)).difference(al)) == 0]

#вероятности каждого слова
spam_d = {v:spam_l.count(v) / len(spam_l) for v in set(spam_l)}
ham_d = {i:ham_l.count(i) / len(ham_l) for i in set(ham_l)}

#доли spam и ham в тренировочных данных
coef_spam = len(train[train['v1']=='spam']) / len(train)
coef_ham = len(train[train['v1']=='ham']) / len(train)

#предсказывание категории - классификатор
def predict(text):
    val_spam, val_ham = 0, 0
    for i in text:
        if i in spam_d:
            val_spam += abs(log(spam_d[i]))
        if i in ham_d:
            val_ham += abs(log(ham_d[i]))
    val_spam += abs(log(coef_spam))
    val_ham += abs(log(coef_ham))
    return val_spam, val_ham

#тестирование модели
def check_row(test):
    result = []
    for _, i in test.iterrows():
        text_check = [i.lower().strip(p) for i in i['v2'].split() if i.lower().strip(p) \
                      not in incons and len(i.strip(p)) >= delete \
                      and len(set(i.strip(p)) & set(p)) == 0 and len(set(i.strip(p)).difference(al)) == 0]
        val_spam, val_ham = predict(text_check)
        answer = 'spam' if val_spam > val_ham else 'ham'
        result.append(1 if i['v1'] == answer else 0)
    return result.count(1), result.count(0)

right, wrong = check_row(test)
print(f'Forecast accuracy = {round(right/(right + wrong) * 100)} %')

Persent right answer = 84 %
